# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [ ]:
pip install catboost

In [189]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

In [190]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [240]:
class DecisionTreeLeaf:
    def __init__(self, y):
        self.y = None
        self.leaf_prediction = {}
        classes = np.unique(y)
        for key in classes:
            self.leaf_prediction[key] = len(np.where(y == key)) / len(y)
        # self.y = max(self.leaf_prediction, key=self.leaf_prediction.get)
        self.y = stats.mode(y)[0][0]


class DecisionTreeNode:
    def __init__(self, split_feature: int, 
                 left: Union['DecisionTreeNode', DecisionTreeLeaf], 
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_feature = split_feature
        self.left = left
        self.right = right


In [241]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=100, min_samples_leaf=30, max_features="auto"):
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        if max_features == 'auto':
            self.max_features = int(np.sqrt(X.shape[1]))
        else:
            self.max_features = max_features
        self.criterion = gini if criterion == 'gini' else entropy
        mask = np.random.randint(0, len(y), size=len(y))   
        mask_out = [i for i in np.arange(len(y)) if i not in mask] 
        self.X = X[mask]
        self.y = y[mask]
        self.X_out_of_bag = X[mask_out]
        self.y_out_of_bag = y[mask_out]   


    def split(self, X, y):
        n, m = X.shape
        best_gain, best_feature = -100, -1
        sample_features = np.random.choice(m, size=self.max_features, replace=False)
        for feature in sample_features:
            mask_left = X[:, feature] == 0
            mask_right = X[:, feature] == 1
            left = y[mask_left]
            right = y[mask_right]
            if len(left) < self.min_samples_leaf or len(right) < self.min_samples_leaf:
                continue
            current_gain = gain(left, right, self.criterion)
            if current_gain > best_gain:
                best_feature, best_gain = feature, current_gain
        return best_feature


    def build(self, X: np.ndarray, y: np.ndarray, depth: int):
        if depth > self.max_depth or len(np.unique(y)) == 1:
            return DecisionTreeLeaf(y)
        depth += 1   
        best_feature = self.split(X, y)
        if best_feature == -1:
          return DecisionTreeLeaf(y)
        mask_left = X[:, best_feature] == 0
        mask_right = X[:, best_feature] == 1
        left = self.build(X[mask_left], y[mask_left], depth)
        right = self.build(X[mask_right], y[mask_right], depth)
        return DecisionTreeNode(best_feature, left, right)


    def recursion(self, X, node):
        if isinstance(node, DecisionTreeLeaf):
            return np.full(X.shape[0], node.y)
        res = np.empty(X.shape[0], dtype=np.object)
        mask = X[:, node.split_feature] == 0
        res[mask] = self.recursion(X[mask], node.left)
        res[~mask] = self.recursion(X[~mask], node.right)
        return res

    def fit(self) -> NoReturn:
        self.root = self.build(self.X, self.y, depth=0)
    
    def predict(self, X : np.ndarray) -> list:
        return self.recursion(X, self.root)


    def predict_out_of_bag(self) -> np.ndarray:
        y_pred = self.predict(self.X_out_of_bag)
        accuracy = np.mean(y_pred == self.y_out_of_bag)
        features = self.X_out_of_bag.shape[1]
        error = np.zeros(features)
        for i in range(features):
            X_ = self.X_out_of_bag.copy()
            X_[:, i] = np.random.permutation(X_[:, i])
            y_ = self.predict(X_)
            acc_ = np.mean(y_ == self.y_out_of_bag)
            error[i] = abs(accuracy - acc_)
        return error


### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [242]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=100, min_samples_leaf=30, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.estimators = np.arange(n_estimators)
    
    def fit(self, X, y):
        self.trees = []
        for i in self.estimators:
            name = 'DecisionTree' + str(i)
            name =  (DecisionTree(X, y, criterion=self.criterion, 
                                  max_depth=self.max_depth, 
                                  min_samples_leaf=self.min_samples_leaf, 
                                  max_features=self.max_features))
            name.fit()
            self.trees.append(name)

    
    def predict(self, X):
        predictions = [[] for _ in range(self.n_estimators)]
        for i, instance in enumerate(self.trees):
            predictions[i] = instance.predict(X)
        res = stats.mode(np.array(predictions), axis=0)[0][0]
        return res.reshape(-1)


    def out_of_bag(self):
        n_errors = [[] for _ in range(self.n_estimators)]    
        for j, instance in enumerate(self.trees):
            n_errors[j] = instance.predict_out_of_bag()
        return np.mean(np.array(n_errors), axis=0)
      

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [243]:
def feature_importance(rfc):
    importance = rfc.out_of_bag()
    return importance

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [244]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.00511895 0.00460624 0.16411932 0.16351796 0.32871575 0.0055671 ]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [245]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [246]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [247]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)
    

In [251]:
X, y_age, y_sex, features = read_dataset("/content/drive/MyDrive/Colab Notebooks/ML/vk.csv")

X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [252]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.6897856242118537
Most important features:
1. 4ch
2. ovsyanochan
3. mudakoff
4. rhymes
5. styd.pozor
6. dayvinchik
7. pravdashowtop
8. rapnewrap
9. tumblr_vacuum
10. iwantyou
11. pixel_stickers
12. reflexia_our_feelings
13. bot_maxim
14. girlmeme
15. memeboizz
16. leprum
17. i_d_t
18. femalemem
19. xfilm
20. iloveu01


#### Пол

In [253]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8310214375788146
Most important features:
1. 40kg
2. girlmeme
3. 9o_6o_9o
4. i_d_t
5. modnailru
6. zerofat
7. cook_good
8. femalemem
9. mudakoff
10. sh.cook
11. psy.people
12. rapnewrap
13. academyofman
14. bon
15. reflexia_our_feelings
16. igm
17. be.beauty
18. be.women
19. 4ch
20. thesmolny


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [180]:
X, y = synthetic_dataset(1000)
catboost_model = CatBoostClassifier(loss_function="MultiClass", verbose=False, random_seed=1000)
catboost_model.fit(X, y)
y_pred = catboost_model.predict(X)
print("Accuracy:", accuracy_score(y_pred,y))
print("Importance:", catboost_model.feature_importances_)

Accuracy: 1.0
Importance: [1.36787893e-02 6.52116243e-03 2.78813422e+01 2.77952182e+01
 4.42948078e+01 8.43188593e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [179]:
X, y_age, y_sex, features = read_dataset("/content/drive/MyDrive/Colab Notebooks/ML/vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [183]:
catboost_model = CatBoostClassifier(custom_loss=['Accuracy'], random_seed=1000, logging_level='Silent', loss_function='MultiClass')
catboost_model.fit(X_train, y_age_train, eval_set=(X_eval, y_age_eval))
y_pred = catboost_model.predict(X_test)
print("Accuracy:", accuracy_score(y_pred, y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(catboost_model.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7414880201765448
Most important features:
1. ovsyanochan
2. mudakoff
3. styd.pozor
4. 4ch
5. dayvinchik
6. rhymes
7. rapnewrap
8. leprum
9. fuck_humor
10. xfilm


#### Пол

In [184]:
catboost_model = CatBoostClassifier(custom_loss=['Accuracy'], random_seed=1000, logging_level='Silent', loss_function='MultiClass')
catboost_model.fit(X_train, y_sex_train, eval_set=(X_eval, y_sex_eval))
y_pred = catboost_model.predict(X_test)
print("Accuracy:", accuracy_score(y_pred, y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(catboost_model.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8814627994955864
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. rapnewrap
5. modnailru
6. femalemem
7. igm
8. i_d_t
9. academyofman
10. thesmolny
